# Коллокации и Pointwise Mutual Information (PMI)

**Коллокация** - сочетание из двух или более слов, которое:

1. обладает некомпозициональной семантикой.
2. состоит из двух слов, которые вместе значимо часто.

При лингвистическом анализе текста коллокации нам очень нужны по понятным причинам:

* выявить, о чем текст
* найти различия между текстами на схожую тематику
* находить упоминания каких-то событий и персон
* и т.д.

Значит, нам нужно уметь выделять их автоматически. Чтобы найти коллокации, состоящие из двух слов, нам нужно выделить такие биграммы, которые встречаются друг с другом значимо часто, по сравнению с их индивидуальными частотами.

Для выделения коллокаций существуют разные метрики. Мы познакомимся с одной из них, которая называется **[Pointwise Mutual Information (PMI)](https://en.wikipedia.org/wiki/Pointwise_mutual_information)**. Формула рассчета PMI - 

$$ pmi = log \frac{{p(x,y)}}{p(x) \cdot p(y)} $$

* x и y - слова, входящие в биграмму;
* p(x,y) - вероятность появления биграммы x + y;
* p(x) и p(y) -- вероятность появления каждого из элементов биграммы в отдельности.

Значит, чтобы посчитать PMI, нам нужно:

* взять текст
* сделать частотный список всех слов в тексте
* найти вероятность появления каждого слова в тексте, разделив его частотность на общее количество слов
* сделать частотный список всех биграмм в тексте
* найти вероятность появления каждой биграммы в тексте, разделив её частотность на общее количество биграмм (а сколько их?)
* найти pmi по формуле
* биграммы с наибольшим pmi - коллокации в этом тексте.

Давайте попробуем:

In [1]:
import re
from math import log

punct = '[.,!«»?&@"$\[\]\(\):;%#&\'—-]'

def preprocessing(text):
    text_wo_punct = re.sub(punct, '', text.lower())
    words = text_wo_punct.strip().split()
    return words

In [3]:
with open('../news.txt', 'r', encoding='utf-8') as f:
    words = preprocessing(f.read())

word_freq = {}
for word in words:
    if word in word_freq:
        word_freq[word] += 1
    else:
        word_freq[word] = 1

bigrams = []
for ind in range(1, len(words) - 1):
    bigrams.append(' '.join([words[ind - 1], words[ind]]))
    
bigram_freq = {}
for b in bigrams:
    if b in bigram_freq:
        bigram_freq[b] += 1
    else:
        bigram_freq[b] = 1

In [4]:
def count_pmi(x, y):
    p_xy = bigram_freq[' '.join([x, y])]
    p_x, p_y = word_freq[x]/len(word_freq), word_freq[y]
    pmi = log(p_xy/(p_x * p_y))
    return pmi

In [5]:
pmi = {}
for bigr in bigrams:
    x, y = bigr.split()
    pmi[bigr] = count_pmi(x, y)

i = 0
for bigram in sorted(pmi, key = lambda m: -pmi[m]):
    if i > 100:
        break
    print(bigram, pmi[bigram])
    i += 1

сложную внешнеполитическую 10.229765142747272
называемом одноканальном 10.229765142747272
социальному служению 10.229765142747272
часовых поясов 10.229765142747272
de lex 10.229765142747272
установлением конкретной 10.229765142747272
судебным посыльным 10.229765142747272
выбранные банкиагенты 10.229765142747272
материальном вознаграждении 10.229765142747272
татарстан тува 10.229765142747272
обнесена инженерными 10.229765142747272
замгубернатора восстановлен 10.229765142747272
давнее знакомство 10.229765142747272
шевкет кайбуллаев 10.229765142747272
камуфляжная одежда 10.229765142747272
носил непартийный 10.229765142747272
страновыми сирия 10.229765142747272
специ­фические вредные 10.229765142747272
громкая либеральная 10.229765142747272
конвертировала банковские 10.229765142747272
сулейман керимов 10.229765142747272
происходили оспариваемые 10.229765142747272
курсовым разницам 10.229765142747272
трудовому договору 10.229765142747272
рента направлялись 10.229765142747272
тебя накажем 10

Какие слова имеют большой коэффициент pmi, а какие маленький?

С помощью pmi можно также искать биграммы, специфичные для какой-то категории текстов. Для этого нам нужно всего лишь посчитать коэффициент PMI для слова/биграммы и категории.

Например, у нас есть тексты 3 категорий. Посчитаем для слов в этих текстах коэффициент PMI, при этом в формуле X будет словом, а Y - категорией. (X, Y) - сколько раз слово X встретилось в текстах категории Y.

In [15]:
import os
anek = ''
teh = ''
izvest = ''
for root, dirs, files in os.walk('texts'):
    for f in files:
        if 'anekdots' in root:
            num_anek = len(files)
            anek += open(os.path.join(root, f)).read()
        elif 'izvest' in root:
            num_izvest = len(files)
            izvest += open(os.path.join(root, f)).read()
        elif 'teh_mol' in root:
            num_teh = len(files)
            teh += open(os.path.join(root, f)).read()
            
words_anek = preprocessing(anek)
words_teh = preprocessing(teh)
words_izvest = preprocessing(izvest)

words = words_anek + words_teh + words_izvest

In [16]:
def freq_dict(arr):
    dic = {}
    for element in arr:
        if element in dic:
            dic[element] += 1
        else:
            dic[element] = 1
    return dic

In [17]:
corpus_freq = freq_dict(words)
anek_freq = freq_dict(words_anek)
izvest_freq = freq_dict(words_izvest)
teh_freq = freq_dict(words_teh)

In [18]:
def pmi_for_cats(x, y):
    if y == 'anek':
        dic = anek_freq
        num = num_anek
    elif y == 'teh':
        dic = teh_freq
        num = num_teh
    elif y == 'izvest':
        dic = izvest_freq
        num = num_izvest
    p_xy = dic[x]
    p_x, p_y = corpus_freq[x], num
    pmi = log(p_xy/(p_x * p_y))
    return pmi

In [22]:
cat_pmi = {}
i = 0
for word in corpus_freq:
    if i > 100:
        break
    try:
        pmi_anek = pmi_for_cats(word, 'anek')
    except KeyError:
        pmi_anek = 0
    try:
        pmi_teh = pmi_for_cats(word, 'teh')
    except KeyError:
        pmi_teh = 0
    try:
        pmi_izvest = pmi_for_cats(word, 'izvest')
    except KeyError:
        pmi_izvest = 0
    max_pmi = max(pmi_anek, pmi_teh, pmi_izvest)
    if max_pmi == 0:
        continue
    if max_pmi == pmi_anek:
        cat = 'anek'
    elif max_pmi == pmi_teh:
        cat = 'teh'
    elif max_pmi == pmi_izvest:
        cat = 'izvest'
    print(word, cat)
    i += 1

тяжелой teh
дуэль anek
станция teh
концу teh
мирового izvest
целый anek
назначению izvest
народами anek
собственными teh
другой anek
русских anek
признание teh
провели anek
суд anek
какаято anek
планете teh
грандиозная anek
развитию izvest
единицы teh
инструменты anek
бурно teh
словом teh
оставаться izvest
удовлетворения anek
прогресс teh
подтверждение teh
специальной teh
200 anek
цвета anek
третьей anek
оба anek
справиться teh
иностранного anek
20 anek
назвать anek
удачно anek
человек anek
черных teh
советским anek
водах teh
80х teh
предложил anek
слушал anek
теле teh
внутренних izvest
чего anek
участке anek
покупка anek
временем teh
широкие teh
окраине anek
легкие anek
эту anek
приборов teh
совпадает teh
прошлой anek
строгим anek
получения teh
большие anek
дверь anek
подобное teh
открытие teh
напоследок anek
оказывают teh
студенты teh
мое anek
41 anek
великое teh
90 teh
лучшем izvest
балки teh
программ izvest
обе anek
конкретного teh
род anek
взяли anek
разницы anek
хранится teh
дока

## Задания

1. Отфильтровать слова, для которых мы вычисляем категории: убрать короткие и служебные, взять самые частотные.
2. Соотнести биграммы с категориями.